# Regret Matching with Rock, Paper, Scissors
---
### Regret
For a group of players let $s_i$ be the action player by player $i$ and $s_{-i}$ the actions played by the remaining players.
Together, these actions form an action profile $a \in A$.
For all other actions $s_i'$ we could've played, we define the **regret of not playing action $s'_i$** as a difference in utility:  

<center>
$
\begin{align*}
    regret(s_i', a) &= u(s_i', s_{i-1}) - u(a)
\end{align*}
$
</center>  

For example, if we play scissors and our opponent plays rock, then $a=(scissors, rock)$ and our utility for this play is $u(scissors, rock) = -1$.
We can compute the regret for all of our possible actions to find:  
<center>
$
\begin{align*}
    regret(rock, a) &= u(rock, rock) - u((scissors, rock)) = 0 - (-1) = 1 \\
    regret(paper, a) &= u(paper, rock) - u((scissors, rock)) = 1 - (-1) = 2 \\
    regret(scissors, a) &= u(scissors, rock) - u((scissors, rock)) = -1 - (-1) = 0
\end{align*}
$  
</center>  

Thus we regret not playing "paper" the most, and regret not playing "rock" more than playing "scissors".Note that when $s_i' = s_i$ the regret is zero. 

---
### Regret Matching

Actions that have positive regret is an indicator that we should've chosen these actions to maximize our utility. 
Thus if we track the regret for each action, if we choose actions at random with probability proportional to how positive their regret is, we can hopefully maximize our utility. Actions with negative regret are given zero probability.
Such a weighting is called **regret matching**.  

If we play another game, using the above regrets we play "paper" with probability $\frac{2}{3}$ and "rock" with probability $\frac{1}{3}$. Suppose we play "paper" while our opponent plays "scissors". 
Thus our regret for this game is:
<center>
$
\begin{align*}
    regret(rock, a) &= u(rock, paper) - u((paper, scissors)) = -1 - (-1) = 0 \\
    regret(paper, a) &= u(paper, paper) - u((paper, scissors)) = 0 - (-1) = 1 \\
    regret(scissors, a) &= u(scissors, paper) - u((paper, scissors)) = 1 - (-1) = 2
\end{align*}
$  
</center>  

If we add these regrets to our previous regrets, we can compute the **cumulative regrets** of $(1,3,2)$ respectively, which is normalized to $(\frac{1}{6},\frac{3}{6},\frac{2}{6})$.
These normalized weights form a mixed-strategy that can be used for the next game.

---

### Example RPS:
We given an example of regret matching against an opponent that plays rock slightly more than paper or scissors.
We initialize the cumulative regret of our agent to 0.
Over several iterations, our agent will choose rock, paper, or scissors with a probability proportional to their cummulative regret via a strategy porfile. We then update their regrets using this action and repeat.  

Regret Matching Algorithm:  
`  
Initialize cummulative regret to 0
For some number of training iterations:
    - Use cummulative regret to define strategy profile
    - Add strategy profile to cummulative strategy profile (will use average after training.)
    - Agent selects action according to strategy profile.
    - Compute agent's regret, given opponents action.
    - Update cummulative regret.
Normalize cummulative strategy profile by number of training iterations.
Return normalized strategy profile. 
`

In [1]:
import random as rd 
import numpy as np


In [4]:
class RPS:
    
    ROCK, PAPER, SCISSORS = 0, 1, 2
    N_ACTIONS = 3
    # Payoff matrix, First index is our agent's choice, Second the opponents choice
    PM = np.array([[0, -1, 1], [1, 0, -1], [-1, 1, 0]])

    def __init__(self, oppo_strat=np.array([1/3, 1/3, 1/3])):
        self.reset(oppo_strat)
        
    def reset(self, oppo_strat):
        self.regret_sum = np.array([0, 0, 0])
        self.strategy_sum = np.array([0, 0, 0])

        self.agent_strategy = np.array([0, 0, 0])
        self.oppo_strategy = oppo_strat
        
    def set_agent_strategy(self):
        # Set agent's strategy proportional to current regret, or uniform if all are non positive.
        normalize_factor = 0
        for a in range(self.N_ACTIONS):
            self.agent_strategy[a] = self.regret_sum[a] if self.regret_sum[a] > 0 else 0
            normalize_factor += self.agent_strategy[a]
            
        for a in range(self.N_ACTIONS):
            if (normalize_factor > 0):
                self.agent_strategy[a] /= normalize_factor
            else:
                self.agent_strategy[a] = 1 / self.N_ACTIONS
                
    def update_running_strategy(self):
        self.strategy_sum += self.agent_strategy
        
    def get_action(self, strategy):
        
        # Given strategy, generate rdnum in [0,1], return action corresponding to bin this number falls into. 
        r = rd.uniform(0,1)

        if r < strategy[0]:
            return self.ROCK
        elif r < strategy[1]+strategy[0]:
            return self.PAPER
        else:
            return self.SCISSORS
        
#     def update_regret(self, agent_action, oppo_action):
        
        
    def train(self, n_epochs):
        for epoch in range(1, n_epochs + 1):
            self.set_agent_strategy()
            self.update_running_strategy()
            agent_action = self.get_action(self.agent_strategy)
            oppo_action = self.get_action(self.oppo_strategy)
            for i in range(self.N_ACTIONS):
                self.regret_sum[i] += self.PM[i][agent_action] - self.PM[agent_action][oppo_action]
            print(self.regret_sum)
        
            if epoch % 500 == 0:
                print(f'Epoch {epoch}. Average strategy: {self.strategy_sum / epoch}')
        
        

In [5]:
rps = RPS([0.8, 0.1, 0.1])
rps.train(10_000)

[2 0 1]
[ 2 -2  0]
[ 2 -1 -1]
[ 2  0 -2]
[ 2  1 -3]
[ 4  1 -2]
[ 6  1 -1]
[8 1 0]
[10  1  1]
[12  1  2]
[14  1  3]
[16  1  4]
[16 -1  3]
[17 -2  3]
[19 -2  4]
[21 -2  5]
[23 -2  6]
[25 -2  7]
[27 -2  8]
[28 -3  8]
[30 -3  9]
[32 -3 10]
[33 -4 10]
[33 -6  9]
[35 -6 10]
[36 -7 10]
[38 -7 11]
[40 -7 12]
[42 -7 13]
[42 -9 12]
[44 -9 13]
[46 -9 14]
[48 -9 15]
[50 -9 16]
[52 -9 17]
[54 -9 18]
[56 -9 19]
[58 -9 20]
[60 -9 21]
[62 -9 22]
[64 -9 23]
[66 -9 24]
[68 -9 25]
[70 -9 26]
[ 71 -10  26]
[ 73 -10  27]
[ 75 -10  28]
[ 77 -10  29]
[ 79 -10  30]
[ 81 -10  31]
[ 83 -10  32]
[ 85 -10  33]
[ 87 -10  34]
[ 89 -10  35]
[ 91 -10  36]
[ 93 -10  37]
[ 95 -10  38]
[ 97 -10  39]
[ 99 -10  40]
[101 -10  41]
[103 -10  42]
[105 -10  43]
[107 -10  44]
[109 -10  45]
[111 -10  46]
[113 -10  47]
[115 -10  48]
[116 -11  48]
[118 -11  49]
[120 -11  50]
[122 -11  51]
[124 -11  52]
[126 -11  53]
[127 -12  53]
[129 -12  54]
[131 -12  55]
[132 -13  55]
[134 -13  56]
[136 -13  57]
[138 -13  58]
[140 -13  59]
[142

[1184 -191  492]
[1185 -192  492]
[1187 -192  493]
[1189 -192  494]
[1189 -194  493]
[1191 -194  494]
[1193 -194  495]
[1195 -194  496]
[1197 -194  497]
[1199 -194  498]
[1201 -194  499]
[1203 -194  500]
[1205 -194  501]
[1207 -194  502]
[1209 -194  503]
[1211 -194  504]
[1213 -194  505]
[1215 -194  506]
[1217 -194  507]
[1219 -194  508]
[1221 -194  509]
[1222 -195  509]
[1224 -195  510]
[1226 -195  511]
[1228 -195  512]
[1229 -196  512]
[1230 -197  512]
[1232 -197  513]
[1234 -197  514]
[1236 -197  515]
[1236 -199  514]
[1238 -199  515]
[1240 -199  516]
[1242 -199  517]
[1244 -199  518]
[1246 -199  519]
[1248 -199  520]
[1250 -199  521]
[1252 -199  522]
[1252 -201  521]
[1254 -201  522]
[1256 -201  523]
[1257 -202  523]
[1259 -202  524]
[1261 -202  525]
[1263 -202  526]
[1265 -202  527]
[1267 -202  528]
[1269 -202  529]
[1271 -202  530]
[1273 -202  531]
[1275 -202  532]
[1277 -202  533]
[1279 -202  534]
[1281 -202  535]
[1283 -202  536]
[1283 -204  535]
[1285 -204  536]
[1287 -204  53

[2461 -412 1020]
[2463 -412 1021]
[2464 -413 1021]
[2466 -413 1022]
[2468 -413 1023]
[2470 -413 1024]
[2472 -413 1025]
[2474 -413 1026]
[2476 -413 1027]
[2478 -413 1028]
[2480 -413 1029]
[2482 -413 1030]
[2484 -413 1031]
[2486 -413 1032]
[2488 -413 1033]
[2490 -413 1034]
[2492 -413 1035]
[2494 -413 1036]
[2496 -413 1037]
[2498 -413 1038]
[2500 -413 1039]
[2502 -413 1040]
[2504 -413 1041]
[2506 -413 1042]
[2508 -413 1043]
[2510 -413 1044]
[2512 -413 1045]
[2514 -413 1046]
[2516 -413 1047]
[2518 -413 1048]
[2520 -413 1049]
[2522 -413 1050]
[2523 -414 1050]
[2525 -414 1051]
[2527 -414 1052]
[2529 -414 1053]
[2531 -414 1054]
[2531 -416 1053]
[2533 -416 1054]
[2534 -417 1054]
[2535 -418 1054]
[2537 -418 1055]
[2539 -418 1056]
[2541 -418 1057]
[2543 -418 1058]
[2545 -418 1059]
[2547 -418 1060]
[2549 -418 1061]
[2550 -419 1061]
[2552 -419 1062]
[2554 -419 1063]
[2556 -419 1064]
[2556 -421 1063]
[2558 -421 1064]
[2559 -422 1064]
[2561 -422 1065]
[2563 -422 1066]
[2565 -422 1067]
[2567 -422 106

[3339 -530 1400]
[3341 -530 1401]
[3342 -531 1401]
[3344 -531 1402]
[3344 -533 1401]
[3345 -534 1401]
[3347 -534 1402]
[3349 -534 1403]
[3351 -534 1404]
[3353 -534 1405]
[3355 -534 1406]
[3357 -534 1407]
[3359 -534 1408]
[3361 -534 1409]
[3363 -534 1410]
[3365 -534 1411]
[3367 -534 1412]
[3369 -534 1413]
[3371 -534 1414]
[3373 -534 1415]
[3375 -534 1416]
[3377 -534 1417]
[3379 -534 1418]
[3381 -534 1419]
[3383 -534 1420]
[3385 -534 1421]
[3387 -534 1422]
[3389 -534 1423]
[3391 -534 1424]
[3393 -534 1425]
[3395 -534 1426]
[3397 -534 1427]
[3399 -534 1428]
[3399 -536 1427]
[3401 -536 1428]
[3403 -536 1429]
[3404 -537 1429]
[3406 -537 1430]
[3406 -539 1429]
[3408 -539 1430]
[3410 -539 1431]
[3410 -541 1430]
[3411 -542 1430]
[3412 -543 1430]
[3414 -543 1431]
[3416 -543 1432]
[3418 -543 1433]
[3420 -543 1434]
[3422 -543 1435]
[3424 -543 1436]
[3426 -543 1437]
[3428 -543 1438]
[3430 -543 1439]
[3432 -543 1440]
[3432 -545 1439]
[3434 -545 1440]
[3436 -545 1441]
[3438 -545 1442]
[3440 -545 144

[4178 -689 1740]
[4179 -690 1740]
[4181 -690 1741]
[4183 -690 1742]
[4185 -690 1743]
[4187 -690 1744]
[4188 -691 1744]
[4189 -692 1744]
[4191 -692 1745]
[4193 -692 1746]
[4195 -692 1747]
[4197 -692 1748]
[4199 -692 1749]
[4201 -692 1750]
[4203 -692 1751]
[4205 -692 1752]
[4207 -692 1753]
[4209 -692 1754]
[4211 -692 1755]
[4213 -692 1756]
[4215 -692 1757]
[4217 -692 1758]
[4219 -692 1759]
[4221 -692 1760]
[4223 -692 1761]
[4225 -692 1762]
[4227 -692 1763]
[4229 -692 1764]
[4231 -692 1765]
[4233 -692 1766]
[4235 -692 1767]
[4237 -692 1768]
[4239 -692 1769]
[4241 -692 1770]
[4243 -692 1771]
[4245 -692 1772]
[4247 -692 1773]
[4249 -692 1774]
[4251 -692 1775]
[4252 -693 1775]
[4254 -693 1776]
[4256 -693 1777]
[4258 -693 1778]
[4260 -693 1779]
[4262 -693 1780]
[4264 -693 1781]
[4266 -693 1782]
[4268 -693 1783]
[4270 -693 1784]
[4272 -693 1785]
[4274 -693 1786]
[4275 -694 1786]
[4277 -694 1787]
[4277 -696 1786]
[4279 -696 1787]
[4281 -696 1788]
[4283 -696 1789]
[4285 -696 1790]
[4287 -696 179

...

[13571 -2262  5650]
[13573 -2262  5651]
[13575 -2262  5652]
[13575 -2264  5651]
[13577 -2264  5652]
[13579 -2264  5653]
[13581 -2264  5654]
[13583 -2264  5655]
[13585 -2264  5656]
[13587 -2264  5657]
[13589 -2264  5658]
[13591 -2264  5659]
[13593 -2264  5660]
[13593 -2266  5659]
[13595 -2266  5660]
[13597 -2266  5661]
[13599 -2266  5662]
[13600 -2267  5662]
[13602 -2267  5663]
[13604 -2267  5664]
[13606 -2267  5665]
[13606 -2269  5664]
[13608 -2269  5665]
[13610 -2269  5666]
[13612 -2269  5667]
[13612 -2271  5666]
[13614 -2271  5667]
[13616 -2271  5668]
[13616 -2273  5667]
[13616 -2275  5666]
[13618 -2275  5667]
[13620 -2275  5668]
[13622 -2275  5669]
[13624 -2275  5670]
[13626 -2275  5671]
[13626 -2277  5670]
[13628 -2277  5671]
[13630 -2277  5672]
[13630 -2279  5671]
[13632 -2279  5672]
[13634 -2279  5673]
[13636 -2279  5674]
[13638 -2279  5675]
[13640 -2279  5676]
[13642 -2279  5677]
[13644 -2279  5678]
[13646 -2279  5679]
[13648 -2279  5680]
[13650 -2279  5681]
[13652 -2279  5682]


[14417 -2414  5997]
[14417 -2416  5996]
[14419 -2416  5997]
[14421 -2416  5998]
[14423 -2416  5999]
[14425 -2416  6000]
[14427 -2416  6001]
[14427 -2418  6000]
[14429 -2418  6001]
[14431 -2418  6002]
[14433 -2418  6003]
[14435 -2418  6004]
[14437 -2418  6005]
[14439 -2418  6006]
[14439 -2420  6005]
[14441 -2420  6006]
[14443 -2420  6007]
[14445 -2420  6008]
[14447 -2420  6009]
[14448 -2421  6009]
[14450 -2421  6010]
[14452 -2421  6011]
[14454 -2421  6012]
[14454 -2423  6011]
[14456 -2423  6012]
[14458 -2423  6013]
[14460 -2423  6014]
[14462 -2423  6015]
[14464 -2423  6016]
[14466 -2423  6017]
[14466 -2425  6016]
[14468 -2425  6017]
[14470 -2425  6018]
[14472 -2425  6019]
[14474 -2425  6020]
[14476 -2425  6021]
[14478 -2425  6022]
[14480 -2425  6023]
[14480 -2427  6022]
[14482 -2427  6023]
[14484 -2427  6024]
[14486 -2427  6025]
[14488 -2427  6026]
[14490 -2427  6027]
[14492 -2427  6028]
[14494 -2427  6029]
[14496 -2427  6030]
[14498 -2427  6031]
[14498 -2429  6030]
[14500 -2429  6031]


[15259 -2568  6341]
[15261 -2568  6342]
[15263 -2568  6343]
[15265 -2568  6344]
[15266 -2569  6344]
[15266 -2571  6343]
[15268 -2571  6344]
[15270 -2571  6345]
[15272 -2571  6346]
[15274 -2571  6347]
[15276 -2571  6348]
[15276 -2573  6347]
[15278 -2573  6348]
[15280 -2573  6349]
[15282 -2573  6350]
[15284 -2573  6351]
[15286 -2573  6352]
[15288 -2573  6353]
[15290 -2573  6354]
[15292 -2573  6355]
[15294 -2573  6356]
[15296 -2573  6357]
[15296 -2575  6356]
[15298 -2575  6357]
[15300 -2575  6358]
[15302 -2575  6359]
[15304 -2575  6360]
[15304 -2577  6359]
[15306 -2577  6360]
[15306 -2579  6359]
[15308 -2579  6360]
[15310 -2579  6361]
[15312 -2579  6362]
[15314 -2579  6363]
[15316 -2579  6364]
[15318 -2579  6365]
[15320 -2579  6366]
[15322 -2579  6367]
[15324 -2579  6368]
[15325 -2580  6368]
[15327 -2580  6369]
[15329 -2580  6370]
[15331 -2580  6371]
[15333 -2580  6372]
[15335 -2580  6373]
[15336 -2581  6373]
[15338 -2581  6374]
[15340 -2581  6375]
[15340 -2583  6374]
[15342 -2583  6375]


[16119 -2706  6702]
[16119 -2708  6701]
[16121 -2708  6702]
[16123 -2708  6703]
[16125 -2708  6704]
[16127 -2708  6705]
[16129 -2708  6706]
[16131 -2708  6707]
[16133 -2708  6708]
[16135 -2708  6709]
[16137 -2708  6710]
[16139 -2708  6711]
[16141 -2708  6712]
[16143 -2708  6713]
[16145 -2708  6714]
[16147 -2708  6715]
[16149 -2708  6716]
[16151 -2708  6717]
[16153 -2708  6718]
[16155 -2708  6719]
[16157 -2708  6720]
[16159 -2708  6721]
[16160 -2709  6721]
[16162 -2709  6722]
[16164 -2709  6723]
[16166 -2709  6724]
[16167 -2710  6724]
[16169 -2710  6725]
[16171 -2710  6726]
[16173 -2710  6727]
[16175 -2710  6728]
[16177 -2710  6729]
[16179 -2710  6730]
[16181 -2710  6731]
[16183 -2710  6732]
[16185 -2710  6733]
[16187 -2710  6734]
[16189 -2710  6735]
[16191 -2710  6736]
[16193 -2710  6737]
[16195 -2710  6738]
[16197 -2710  6739]
[16198 -2711  6739]
[16200 -2711  6740]
[16202 -2711  6741]
[16204 -2711  6742]
[16206 -2711  6743]
[16208 -2711  6744]
[16210 -2711  6745]
[16212 -2711  6746]


[16984 -2837  7069]
[16986 -2837  7070]
[16988 -2837  7071]
[16989 -2838  7071]
[16991 -2838  7072]
[16993 -2838  7073]
[16995 -2838  7074]
[16997 -2838  7075]
[16999 -2838  7076]
[17000 -2839  7076]
[17002 -2839  7077]
[17002 -2841  7076]
[17004 -2841  7077]
[17006 -2841  7078]
[17008 -2841  7079]
[17010 -2841  7080]
[17012 -2841  7081]
[17014 -2841  7082]
[17016 -2841  7083]
[17016 -2843  7082]
[17018 -2843  7083]
[17020 -2843  7084]
[17021 -2844  7084]
[17023 -2844  7085]
[17023 -2846  7084]
[17023 -2848  7083]
[17025 -2848  7084]
[17027 -2848  7085]
[17029 -2848  7086]
[17031 -2848  7087]
[17033 -2848  7088]
[17035 -2848  7089]
[17036 -2849  7089]
[17038 -2849  7090]
[17040 -2849  7091]
[17042 -2849  7092]
[17044 -2849  7093]
[17044 -2851  7092]
[17044 -2853  7091]
[17046 -2853  7092]
[17048 -2853  7093]
[17048 -2855  7092]
[17050 -2855  7093]
[17052 -2855  7094]
[17054 -2855  7095]
[17056 -2855  7096]
[17058 -2855  7097]
[17058 -2857  7096]
[17060 -2857  7097]
[17062 -2857  7098]
